In [1]:
# !wget https://huggingface.co/datasets/mesolitica/embedding-pair-mining/resolve/main/b.cari.com.my-dataset-bge-test.jsonl

In [2]:
# !pip3 install openai

In [3]:
# we use OpenAI Azure, replace it with standard OpenAI

In [4]:
import os

os.environ['OPENAI_API_KEY'] = ''

In [5]:
import json
import openai
from tqdm import tqdm

In [6]:
with open('b.cari.com.my-dataset-bge-test.sample.json') as fopen:
    rev_data = json.load(fopen)

In [8]:
vectors = {}
for k, v in tqdm(rev_data.items()):
    try:
        o = openai.Embedding.create(input=k, engine=engine)
        vectors[k] = o['data'][0]['embedding']
    except:
        pass

100%|██████████| 18755/18755 [1:20:46<00:00,  3.87it/s]


In [8]:
no_string = {no: k for no, (k, v) in enumerate(vectors.items())}
string_no = {v: k for k, v in no_string.items()}
no_string[0]

'Setiap orang harus memiliki hati yang wangi. Semua hati harus busuk.'

In [9]:
no_string[1]

'Apis Rosdi Tampoi atau Tanjung Rembutan? Katanya Tanjung Rambutan adalah adik perempuan.'

In [9]:
with open('openai-b.cari.com.my.vectors.json', 'w') as fopen:
    json.dump(vectors, fopen)

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [44]:
line = 0
positive, negative = [], []
with open('b.cari.com.my-dataset-bge-test.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        query = l['query'].strip()
        v_query = vectors.get(query)
        if v_query is None:
            continue
        v_query = np.array(v_query).reshape((1, -1))
        for s in l['pos']:
            v_s = vectors.get(s.strip())
            if v_s is None:
                continue
            v_s = np.array(v_s).reshape((1, -1))
            positive.append(cosine_similarity(v_query, v_s)[0, 0])
        
        for s in l['neg']:
            v_s = vectors.get(s.strip())
            if v_s is None:
                continue
            v_s = np.array(v_s).reshape((1, -1))
            negative.append(1 - cosine_similarity(v_query, v_s)[0, 0])
            
        line += 1
        if line >= 1000:
            break

999it [00:07, 126.73it/s]


In [46]:
np.mean(positive), np.mean(negative)

(0.8729225971201091, 0.27480777421889363)

In [11]:
np_vectors = np.array(list(vectors.values()))

In [15]:
tops = {
    1: 0,
    3: 0,
    5: 0,
    10: 0,
}
total = 0
with open('b.cari.com.my-dataset-bge-test.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        query = l['query'].strip()
        query_no = string_no.get(query)
        if query_no is None:
            continue
        for s in l['pos']:
            s = s.strip()
            v_s = vectors.get(s)
            s_no = string_no.get(s)
            if v_s is None:
                continue
            v_s = np.array(v_s).reshape((1, -1))
            argsort = np.argsort(cosine_similarity(v_s, np_vectors)[0])[::-1]
            for k in tops.keys():
                if s_no in argsort[:k]:
                    k_ = k + 1
                else:
                    k_ = k
                if query_no in argsort[:k_]:
                    tops[k] += 1
            total += 1

9541it [09:25, 16.88it/s] 


In [18]:
tops, total

({1: 1010, 3: 1994, 5: 2218, 10: 2435}, 3194)

In [19]:
for k, v in tops.items():
    print(k, v / total)

1 0.31621790857858484
3 0.6242955541640576
5 0.6944270507201001
10 0.7623669380087664
